**Importamos las librerías necesarias**

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

**Cargamos nuestras bases de datos**

In [2]:
cg = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
dg = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")
rg = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv")

**Recordemos que:**
- cg: Tabla de casos confirmados a nivel global.
- dg: Tabla de muertes a nivel global.
- rg: Tabla de recuperados a nivel global.